# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [2]:
data_dir = '/Users/marjimbel/PycharmProjects/pytorch_practice/notebooks/Cat_Dog_data'
image_size = 224

# TODO: Define transforms for the training data and testing data

train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size-1),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size-1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [3]:
model = models.densenet121(pretrained=True)
model

/Users/marjimbel/PycharmProjects/pytorch_practice/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/marjimbel/PycharmProjects/pytorch_practice/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /Users/marjimbel/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
76.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [4]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [11]:
import time
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

#device = torch.device("mps")

True
True


In [12]:
for device in ['cpu', 'mps']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

#    device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = mps; Time per batch: 0.205 seconds
Device = mps; Time per batch: 0.205 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [23]:
def validation(model, inputs, criterion):

    accuracy = 0
    test_loss = 0

    device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:

            inputs, labels = inputs.to(device), labels.to(device)
            output = model.forward(inputs)
            test_loss += criterion(output, labels).item()

            ## Calculating the accuracy
            # Model's output is log-softmax, take exponential to get the probabilities
            ps = torch.exp(output)
            # Class with highest probability is our predicted class, compare with true label
            equality = (labels.data == ps.max(1)[1])
            # Accuracy is number of correct predictions divided by all predictions, just take the mean
            accuracy += equality.type_as(torch.FloatTensor()).mean()

    model.train()
    return test_loss, accuracy

In [24]:
## TODO: Use a pretrained model to classify the cat and dog images
device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
model.to(device)

inputs, _ = next(iter(trainloader))
features_dim = inputs.view(inputs.shape[0],-1).shape[1]

epochs = 1
steps = 0
running_loss = 0

for e in range(0,epochs):
    for ii, (inputs, labels) in enumerate(trainloader):
        steps+=1
        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        running_loss += loss.item()

        optimizer.step()

        if steps % 10 == 0:
            # Model in inference mode, dropout is off
            model.eval()

            #Turn off gradients for validation, will speed up inference
            with torch.no_grad():
               test_loss, accuracy = validation(model, testloader, criterion)

            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/10),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

            running_loss = 0

            # Make sure dropout and grads are on for training
            model.train()


Epoch: 1/1..  Training Loss: 0.069..  Test Loss: 0.058..  Test Accuracy: 0.979
Epoch: 1/1..  Training Loss: 0.112..  Test Loss: 0.097..  Test Accuracy: 0.962
Epoch: 1/1..  Training Loss: 0.122..  Test Loss: 0.110..  Test Accuracy: 0.961
Epoch: 1/1..  Training Loss: 0.093..  Test Loss: 0.070..  Test Accuracy: 0.973
Epoch: 1/1..  Training Loss: 0.093..  Test Loss: 0.087..  Test Accuracy: 0.967
Epoch: 1/1..  Training Loss: 0.084..  Test Loss: 0.051..  Test Accuracy: 0.982
Epoch: 1/1..  Training Loss: 0.075..  Test Loss: 0.057..  Test Accuracy: 0.980
Epoch: 1/1..  Training Loss: 0.058..  Test Loss: 0.070..  Test Accuracy: 0.973
Epoch: 1/1..  Training Loss: 0.080..  Test Loss: 0.049..  Test Accuracy: 0.981
Epoch: 1/1..  Training Loss: 0.067..  Test Loss: 0.058..  Test Accuracy: 0.975
Epoch: 1/1..  Training Loss: 0.068..  Test Loss: 0.051..  Test Accuracy: 0.982
Epoch: 1/1..  Training Loss: 0.070..  Test Loss: 0.060..  Test Accuracy: 0.978
Epoch: 1/1..  Training Loss: 0.063..  Test Loss: 0.0